### Project 3: Implementing a Simple Recurrent Neural Network (RNN)

#### Environment Setup

In [4]:
pip install tensorflow-datasets

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
   ------ --------------------------------- 0.9/5.1 MB 27.4 MB/s eta 0:00:01
   --------------------- ------------------ 2.7/5.1 MB 35.1 MB/s eta 0:00:01
   ------------------------------------ --- 4.7/5.1 MB 42.8 MB/s eta 0:00:01
   ---------------------------------------  5.1/5.1 MB 32.6 MB/s eta 0:00:01
   ---------------------------------------- 5.1/5.1 MB 29.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/152.4 kB ? eta -:--:--
   ---------------------------------------- 152.4/152.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/101.3 kB ? eta -:--:--
   ---------------------------------------- 101.3/101.3 kB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/113.6 kB ? eta -:--:--
   ---------------------------------------- 113.6/113.6 kB 6.9 MB/s eta 0:00:0

  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install tensorflow numpy matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### GPU Setup

In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

#### Data Loader and Preprocessing

In [11]:
# Load the dataset
with open('text8', 'r') as file:
    text = file.read()

# Define the split ratios
train_ratio = 0.9
val_ratio = 0.05
test_ratio = 0.05

# Calculate the split indices
train_end = int(len(text) * train_ratio)
val_end = train_end + int(len(text) * val_ratio)

# Split the dataset
train_text = text[:train_end]
val_text = text[train_end:val_end]
test_text = text[val_end:]

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import numpy as np

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([train_text])
total_words = len(tokenizer.word_index) + 1

# Create sequences from the text
def create_sequences(text, tokenizer, max_length):
    input_sequences = []
    for i in range(max_length, len(text)):
        seq = text[i - max_length:i + 1]
        encoded_seq = tokenizer.texts_to_sequences([seq])[0]
        input_sequences.append(encoded_seq)
    return input_sequences

# Define maximum sequence length
max_length = 100

# Create input sequences for train, validation, and test sets
train_sequences = create_sequences(train_text, tokenizer, max_length)
val_sequences = create_sequences(val_text, tokenizer, max_length)
test_sequences = create_sequences(test_text, tokenizer, max_length)

# Convert to numpy arrays
train_sequences = np.array(train_sequences)
val_sequences = np.array(val_sequences)
test_sequences = np.array(test_sequences)

# Split input and target sequences
def split_input_target(sequences):
    input_sequences = sequences[:, :-1]
    target_sequences = sequences[:, -1]
    return input_sequences, target_sequences

x_train, y_train = split_input_target(train_sequences)
x_val, y_val = split_input_target(val_sequences)
x_test, y_test = split_input_target(test_sequences)

# Convert target sequences to one-hot encoding
y_train = to_categorical(y_train, num_classes=total_words)
y_val = to_categorical(y_val, num_classes=total_words)
y_test = to_categorical(y_test, num_classes=total_words)

#### Design the Recurrent Neural Network

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Embedding(total_words, 256, input_length=max_length),
    SimpleRNN(1024, return_sequences=True, stateful=False, recurrent_initializer='glorot_uniform'),
    Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

NameError: name 'info' is not defined

#### Compile and Train the Model

In [ ]:
# Define training parameters
BATCH_SIZE = 64
EPOCHS = 10

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

#### Evaluation and Visualization

In [ ]:
import matplotlib.pyplot as plt

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()